In [2]:
import pandas as pd
import numpy as np
import os
import glob
import re
import pdf2image
import pdfminer
import pytesseract
from pytesseract import Output
from PIL import Image
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import cv2
import io
import tempfile
from pytesseract import image_to_string

In [11]:
def main(inputfile):
    def get_grayscale(image):
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    def remove_noise(image):
        return cv2.medianBlur(image, 3)
    def thresholding(image):
        return cv2.threshold(image, 160, 255, cv2.THRESH_BINARY)[1]
    def dilate(image):
        kernel = np.ones((5, 5), np.uint8)
        return cv2.dilate(image, kernel, iterations=1)
    def erode(image):
        kernel = np.ones((5, 5), np.uint8)
        return cv2.erode(image, kernel, iterations=1)
    def opening(image):
        kernel = np.ones((5, 5), np.uint8)
        return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    def canny(image):
        return cv2.Canny(image, 100, 200)
    def deskew(image):
        coords = np.column_stack(np.where(image > 0))
        angle = cv2.minAreaRect(coords)[-1]
        if angle < -45:
            angle = -(90 + angle)

        else:
            angle = -angle
            (h, w) = image.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
            return rotated
    def match_template(image, template):
        return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

    im = Image.open(inputfile)
    length_x, width_y = im.size
    factor = float(1024.0 / length_x)
    size = int(factor * length_x), int(factor * width_y)
    image_resize = im.resize(size, Image.Resampling.LANCZOS)
    image_resize.save(f"{inputfile}_preprocessed.png", dpi=(300, 300))
    img = cv2.imread(f"{inputfile}_preprocessed.png")
    gray = get_grayscale(img)
    ## Different Modes for image proccessing
    img = gray
    # deskew = deskew(gray)
    # erode = erode(gray)
    # thresh = thresholding(gray)
    # thresh = remove_noise(gray)
    # opening = opening(gray)
    # canny = canny(gray)

In [12]:
if __name__ == "__main__":
    main(inputfile=r'C://Users/Omid roodi/Desktop/OCR_test/Screenshot_20231115_132605_Samsung Notes.jpg')

In [5]:
img = Image.open(r'C://Users/Omid roodi/Desktop/OCR_test/Screenshot_20231115_132605_Samsung Notes.jpg_preprocessed.png')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, threshold_img = cv2.threshold(gray_img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
preprocessed_img = Image.fromarray(threshold_img)

error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
text = pytesseract.image_to_string(preprocessed_img, lang='fas', config='--psm 6 oem 3')

In [21]:
# Open an image file
image_path = r'C://Users/Omid roodi/Desktop/OCR_test/Screenshot_20231115_132605_Samsung Notes.jpg'
img = Image.open(image_path)

# Use pytesseract to do OCR on the image, specifying the language as Persian ('fas')
# Include your custom words file in the configuration
config = '--psm 6 oem 3 user_words C://Users/Omid roodi/Desktop/OCR_test/your_custom_words.txt'
text = pytesseract.image_to_string(img, lang='fas', config=config)

print(text)

۲ ات
۱۸۳
۹۱۷۳۹
۱۷۰۸
۱-۸-۳۷
۶ +(
0۱
۸,۹۲۱,۹۶۰ ۶۳)
(۸,۲۱۲,۲۴۱)
(۵۲,۶۳۲)
(۱۸,۰۷۷,۵۷۶)
۴۰۸,۹۳۴,۹۹۸
(۹۹۹,۸۰۹,۱۰۴)
(۱۳۲۱,۷۱۶,۷۵۲)
(۴۵,۶۲۲,۹۵۱)
(۱۱,۰۷۶,۹۷۸)



In [ ]:
image_path= r'C://Users/Omid roodi/Desktop/OCR_test/Screenshot_20231115_132605_Samsung Notes.jpg'
img = Image.open(image_path)
text= pytesseract.image_to_string(img,lang='fas',config='--psm 6 oem 3')

In [ ]:
lines= text.split('\n')
df = pd.DataFrame(lines)

In [ ]:
df